In [ ]:
!pip install -q diffusers transformers peft accelerate bitsandbytes

In [ ]:
import os
import pandas as pd

df = pd.read_csv('/kaggle/working/metadata.csv', sep=",")

filtered_df = df.sort_values(['clip_score', 'human_score'], ascending=[False, False]).groupby('prompt').first().reset_index()

selected_images = filtered_df['image_path'].tolist()

image_dir = '/kaggle/working/clip_feedback_dataset/images/'

print("Selected images:")
print(selected_images)

existing_files = os.listdir(image_dir)
print("Existing files in image directory:")
print(existing_files)

to_delete = [f for f in existing_files if f not in selected_images]

print("Files that would be deleted:")
print(to_delete)

filtered_df.to_csv('/kaggle/working/clip_feedback_dataset/filtered_metadata.csv', index=False)

In [ ]:
import os
import shutil
filtered_dir = "/kaggle/working/filtered_images"
os.makedirs(filtered_dir, exist_ok=True)

# Copy each selected image to the new directory
for img_path in selected_images:
    full_src_path = os.path.join("/kaggle/working", img_path)
    filename = os.path.basename(img_path)
    dest_path = os.path.join(filtered_dir, filename)
    
    if os.path.exists(full_src_path):
        shutil.copy(full_src_path, dest_path)
    else:
        print(f"WARNING: File not found -> {full_src_path}")

print(f"All selected images copied to: {filtered_dir}")


In [1]:
!wget https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/train_dreambooth_lora.py -O /kaggle/working/train_dreambooth_lora.py

--2025-05-05 10:11:13--  https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/train_dreambooth_lora.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58969 (58K) [text/plain]
Saving to: ‘/kaggle/working/train_dreambooth_lora.py’

/kaggle/working/tra 100%[===================>]  57.59K  --.-KB/s    in 0.01s   

2025-05-05 10:11:13 (4.57 MB/s) - ‘/kaggle/working/train_dreambooth_lora.py’ saved [58969/58969]



In [2]:
!pip uninstall -y diffusers
!pip install git+https://github.com/huggingface/diffusers


Found existing installation: diffusers 0.32.2
Uninstalling diffusers-0.32.2:
  Successfully uninstalled diffusers-0.32.2
  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-mfhp_k29
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-mfhp_k29
  Resolved https://github.com/huggingface/diffusers to commit a674914fd5f45ef7bcec71061aa2fb315ceb3495
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.34.0.dev0-py3-none-any.whl size=3602895 sha256=1d642e47bf79eb562312841190b5c1901ac8b601b10d58051fac930459e9be22
  Stored in directory: /tmp/pip-ephem-wheel-cache-xe6imugm/wheels/90/fb/48/a310c271ab42899362ff272062ced42133e5c4c9d0ce77df68
Successfully built diffusers


In [9]:
!accelerate launch train_dreambooth_lora.py \
--pretrained_model_name_or_path=CompVis/stable-diffusion-v1-4 \
--instance_data_dir=/kaggle/input/images \
--output_dir=./lora_sd_output \
--instance_prompt="a photo of a sks person" \
--resolution=512 \
--train_batch_size=1 \
--gradient_accumulation_steps=1 \
--learning_rate=1e-4 \
--lr_scheduler=constant \
--max_train_steps=800 \
--train_text_encoder


2025-05-05 10:21:22.363187: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746440482.386096     231 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746440482.393176     231 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-05 10:21:22.747851: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746440482.771439     230 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746440482.778431     230 cuda_blas.cc:1

In [11]:
!zip -r /kaggle/working/lora_sd_output.zip /kaggle/working/lora_sd_output

updating: kaggle/working/lora_sd_output/ (stored 0%)
updating: kaggle/working/lora_sd_output/pytorch_lora_weights.safetensors (deflated 6%)
updating: kaggle/working/lora_sd_output/logs/ (stored 0%)
updating: kaggle/working/lora_sd_output/logs/dreambooth-lora/ (stored 0%)
updating: kaggle/working/lora_sd_output/logs/dreambooth-lora/1746440492.563865/ (stored 0%)
updating: kaggle/working/lora_sd_output/logs/dreambooth-lora/1746440492.563865/hparams.yml (deflated 52%)
updating: kaggle/working/lora_sd_output/logs/dreambooth-lora/events.out.tfevents.1746440492.065faf2bf3d2.230.0 (deflated 66%)
updating: kaggle/working/lora_sd_output/logs/dreambooth-lora/1746440492.5618944/ (stored 0%)
updating: kaggle/working/lora_sd_output/logs/dreambooth-lora/1746440492.5618944/events.out.tfevents.1746440492.065faf2bf3d2.230.1 (deflated 56%)
updating: kaggle/working/lora_sd_output/checkpoint-500/ (stored 0%)
updating: kaggle/working/lora_sd_output/checkpoint-500/scheduler.bin (deflated 55%)
updating: kagg

In [13]:
from diffusers import StableDiffusionPipeline
from peft import PeftModel
import torch

base_model = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
).to("cuda")

lora_path = "/kaggle/working/lora_sd_output/pytorch_lora_weights.safetensors"  # Replace with your path
base_model.load_lora_weights(lora_path)

prompt = "a futuristic motorcycle in neon city"
image = base_model(prompt).images[0]

image.save("lora_result.png")


2025-05-05 10:48:36.674693: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746442116.697418      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746442116.704009      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]